In [3]:
import ee

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [28]:
# 撮影日リストの取得関数
def get_shootingdate_list(imagecollection):
    shootingdate_list = (
        ee.List(imagecollection.aggregate_array('system:time_start'))
        .map(lambda d : ee.Date(d).format('YYYY-MM-dd'))
        .distinct()
    )
    return shootingdate_list.getInfo()

    # NDVI画像取得関数
def get_ndvi_image(ImageCollection,shootingdate):
    ndvi_image = (
        ImageCollection
        .filterDate(
            ee.Date(shootingdate),
            ee.Date(shootingdate).advance(1,'day')
        )
        .select(['B8','B4'])
        .mean()
        .normalizedDifference(['B8','B4'])
        .rename('ndvi')
        .clip(ee.Geometry.Polygon(coords))
    )
    return ndvi_image

base = [43.75054584331872, 143.67532544734632]
coords_range = 0.001

coords = [[
    [base[1] + coords_range, base[0] + coords_range],
    [base[1] - coords_range, base[0] + coords_range],
    [base[1] - coords_range, base[0] - coords_range],
    [base[1] + coords_range, base[0] - coords_range],
    [base[1] + coords_range, base[0] + coords_range],
]]

# 対象期間（開始）
START_DATE ='2021-06-01'
# 対象期間（終了）
END_DATE = '2021-07-30'
# 雲被覆率フィルタリングの上限値
CLOUD_COVER_RATE = 20

# ImageCollectionの絞り込み
imgcol = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(ee.Geometry.Polygon(coords))
    .filterDate(ee.Date(START_DATE), ee.Date(END_DATE))
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',CLOUD_COVER_RATE))
)

# 撮影日リストの取得
shootingdate_list = get_shootingdate_list(imgcol)
print(shootingdate_list)

# 緯度・経度情報に基づき解析するためCRSはEPSG:4326を指定しておく
for shootingdate in shootingdate_list:
    task = ee.batch.Export.image.toDrive(
        image = get_ndvi_image(imgcol,shootingdate),
        description = 'NDVI_'+shootingdate,
        scale = 10,
        folder = 'ExportGEEimage',
        crs = 'EPSG:4326'
    )
    task.start()

['2021-06-01', '2021-06-06', '2021-06-13', '2021-06-23', '2021-06-26', '2021-06-28', '2021-07-18', '2021-07-23', '2021-07-26', '2021-07-28']


In [32]:
# 画像取得したいポリゴン座標
base = [43.75054584331872, 143.67532544734632]
coords_range = 0.01

coords = [[
    [base[1] + coords_range, base[0] + coords_range],
    [base[1] - coords_range, base[0] + coords_range],
    [base[1] - coords_range, base[0] - coords_range],
    [base[1] + coords_range, base[0] - coords_range],
    [base[1] + coords_range, base[0] + coords_range],
]]

# 対象期間（開始）
START_DATE ='2021-07-01'
# 対象期間（終了）
END_DATE = '2021-7-30'
# 雲被覆率フィルタリングの上限値
CLOUD_COVER_RATE = 30

# ImageCollectionの絞り込み
imgcol = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(ee.Geometry.Polygon(coords))
    .filterDate(ee.Date(START_DATE), ee.Date(END_DATE))
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',CLOUD_COVER_RATE))
)

# ImageCollectionに格納されているImageIDのリスト化と確認
asset_id_list = imgcol.aggregate_array('system:id').getInfo()
print(asset_id_list)

# 画像をGoogleDrive内に出力
for i,asset_id in enumerate(asset_id_list):
    print(i)
    task = ee.batch.Export.image.toDrive(
        image = ee.Image(asset_id_list[i]).select(['TCI_R','TCI_G','TCI_B']),
        region = ee.Geometry.Polygon(coords),
        description = 'Sample'+str(i),
    )
    task.start()

['COPERNICUS/S2_SR_HARMONIZED/20210701T012659_20210701T012656_T54TYP', 'COPERNICUS/S2_SR_HARMONIZED/20210718T011659_20210718T011654_T54TYP', 'COPERNICUS/S2_SR_HARMONIZED/20210721T012659_20210721T012657_T54TYP', 'COPERNICUS/S2_SR_HARMONIZED/20210723T011701_20210723T011656_T54TYP', 'COPERNICUS/S2_SR_HARMONIZED/20210726T012701_20210726T012658_T54TYP', 'COPERNICUS/S2_SR_HARMONIZED/20210728T011659_20210728T011654_T54TYP']
0
1
2
3
4
5
